In [8]:
import pandas as pd
import numpy as np
import glob


In [ ]:
adhd_files = glob.glob("adhd/*.csv")
nonadhd_files = glob.glob("nonadhd/*.csv")

# Combine ADHD data
adhd_dfs = []
for file in adhd_files:
    df = pd.read_csv(file)
    adhd_dfs.append(df)
adhd_cpt = pd.concat(adhd_dfs, ignore_index=True)
adhd_cpt["Group"] = "ADHD"

# Combine Non-ADHD data
nonadhd_dfs = []
for file in nonadhd_files:
    df = pd.read_csv(file)
    nonadhd_dfs.append(df)
nonadhd_cpt = pd.concat(nonadhd_dfs, ignore_index=True)
nonadhd_cpt["Group"] = "No-ADHD"

#Combine into one DataFrame
combined_cpts = pd.concat([adhd_cpt, nonadhd_cpt], ignore_index=True)

In [20]:
#Calculate Averages across each group
group_means = combined_cpts.groupby("Group").mean(numeric_only=True)

#Calculate the Standard deviation across each group
group_stds = combined_cpts.groupby("Group").std(numeric_only=True)

#Calculate deltas
mean_deltas = group_means.loc["ADHD"] - group_means.loc["No-ADHD"]
std_deltas = group_stds.loc["ADHD"] - group_stds.loc["No-ADHD"]

#Build summary table
summary = pd.DataFrame ({
    "ADHD_Mean": group_means.loc["ADHD"],
    "ADHD_SD": group_stds.loc["ADHD"],
    "NoADHD_Mean": group_means.loc["No-ADHD"],
    "NoADHD_SD": group_stds.loc["No-ADHD"],
    "Mean_Delta": mean_deltas,
    "SD_Delta": std_deltas,
})


In [21]:
#Export
summary.to_csv("CPT_Summary.csv")
